In [ ]:
pip install natasha nltk xgboost

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('all')
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, accuracy_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_da

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [ ]:
russian_stopwords = set(stopwords.words("russian"))

In [ ]:
def natasha_lemmatize(text):
    text = text.lower()

    text = re.sub(r'http\S+|www.\S+|[\w\.-]+@[\w\.-]+', ' ', text)

    text = re.sub(r'[^а-яёa-z0-9\s]', ' ', text)

    tokens = word_tokenize(text, language='russian')
    clean_text = " ".join(tokens)

    doc = Doc(clean_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    lemmas = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemma = token.lemma

        if lemma in russian_stopwords:
            continue
        if lemma.isdigit():
            continue
        if len(lemma) < 2:
            continue

        lemmas.append(lemma)

    return " ".join(lemmas)


In [ ]:
mintrade_keywords = {
    "торговл", "интеграц", "экспорт", "импорт",
    "qaztrade", "минторговл", "вто", "антидемпинг",
    "ценовая", "продовольств"
}


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class MinTradeKeywordCounter(BaseEstimator, TransformerMixin):
    def __init__(self, keywords):
        self.keywords = keywords

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        counts = []
        for text in X:

            tokens = text.split()
            count = sum(1 for tok in tokens if any(k in tok for k in self.keywords))
            counts.append(count)
        return np.array(counts).reshape(-1, 1)

In [ ]:
df = pd.read_json("/content/MinistryIntegration.json")

In [ ]:
df

,title,content,ministry
0,Казахстанский павильон открыт в Термезе,Министр торговли и интеграции Республики Казах...,Министерство торговли и интеграции
1,Заместителем министра торговли и интеграции ст...,Постановлением правительства Асет Ильясович Ну...,Министерство торговли и интеграции
2,"В Минторговли считают, что в этом году дефицит...",Министерство торговли и интеграции Казахстана ...,Министерство торговли и интеграции
3,Какова доля казахстанских продуктов на полках ...,Заместитель председателя Комитета торговли Мин...,Министерство торговли и интеграции
4,“Все зависит от ваших возможностей“ - министр ...,Министр торговли и интеграции Арман Шаккалиев ...,Министерство торговли и интеграции
...,...,...,...
397,Первое повышение цен на бензин будет не больше...,"Министр энергетики Алмасадам Саткалиев заявил,...",another ministry
398,Подходящие площадки для строительства второй к...,Министр энергетики Алмасадам Саткалиев сообщил...,another ministry
399,Кудайберген Арымбек назначен вице-министром эн...,Постановлением Правительства Республики Казахс...,another ministry
400,Глава Минэнерго посетил Ленинградскую АЭС,Министр энергетики Казахстана Алмасадам Саткал...,another ministry


In [ ]:
df['text']= df['title']+' '+df['content']

In [ ]:
df.head()

,title,content,ministry,text
0,Казахстанский павильон открыт в Термезе,Министр торговли и интеграции Республики Казах...,Министерство торговли и интеграции,Казахстанский павильон открыт в Термезе Минист...
1,Заместителем министра торговли и интеграции ст...,Постановлением правительства Асет Ильясович Ну...,Министерство торговли и интеграции,Заместителем министра торговли и интеграции ст...
2,"В Минторговли считают, что в этом году дефицит...",Министерство торговли и интеграции Казахстана ...,Министерство торговли и интеграции,"В Минторговли считают, что в этом году дефицит..."
3,Какова доля казахстанских продуктов на полках ...,Заместитель председателя Комитета торговли Мин...,Министерство торговли и интеграции,Какова доля казахстанских продуктов на полках ...
4,“Все зависит от ваших возможностей“ - министр ...,Министр торговли и интеграции Арман Шаккалиев ...,Министерство торговли и интеграции,“Все зависит от ваших возможностей“ - министр ...


In [ ]:
df['label'] = np.where(
    df['ministry'] == 'Министерство торговли и интеграции', 1, 0
)

df['clean_text'] = df['text'].apply(natasha_lemmatize)

X = df['clean_text']
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
models = {
    "LogisticRegression": LogisticRegression(solver='liblinear'),
    "RandomForest": RandomForestClassifier(n_estimators=100),
    "SVC": SVC(),
    "NaiveBayes": MultinomialNB(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

best_pipeline = None
best_model_name = None
best_accuracy = 0.0

In [ ]:
keyword_counter = MinTradeKeywordCounter(mintrade_keywords)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

final_pipeline = Pipeline([
    ('union', FeatureUnion([
        ('tfidf', TfidfVectorizer()),
        ('mintrade_keywords', keyword_counter)
    ])),
    ('clf', LogisticRegression(solver='liblinear'))
])
final_pipeline.fit(X_train, y_train)
y_pred = final_pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\n===== {model_name} =====")
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=['Другое', 'Минторговля']))
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model_name = model_name
    best_pipeline = pipeline

print(f"\nЛучшая модель: {best_model_name} (Accuracy: {best_accuracy:.4f})")
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer




===== XGBoost =====
Accuracy: 0.9382716049382716
              precision    recall  f1-score   support

      Другое       0.91      0.98      0.94        41
 Минторговля       0.97      0.90      0.94        40

    accuracy                           0.94        81
   macro avg       0.94      0.94      0.94        81
weighted avg       0.94      0.94      0.94        81


Лучшая модель: XGBoost (Accuracy: 0.9383)


In [ ]:
new_texts = [
   "Постановлением правительства Асет Ильясович Нусупов назначен на должность вице-министра торговли и интеграции Республики Казахстан. Об этом сообщает пресс-служба министерства.\n\nАсет Нусупов родился в 1983 году в Алматы, окончил Казахский национальный университет им. аль-Фараби и Иллинойский институт технологий (США). Начал карьеру в 2004 году менеджером АО «Национальный инновационный фонд».\n\nВ разное время работал в инвестиционных и финансовых структурах, включая Департамент финансирования венчурных проектов Flagship Ventures (США), Small Enterprise Assistance Funds (США), Инвестиционный фонд Казахстана, представительства Всемирного банка и Азиатского банка развития в Центральной Азии. Также занимал руководящие должности в Совете безопасности Республики Казахстан, где работал с 2010 по 2023 год.\n\nДо назначения на новую должность являлся заместителем заведующего отделом актуальных вопросов безопасности Совета безопасности РК.",
   "Автовладельцы Алматы столкнулись с неожиданными штрафами за объезд припаркованных автомобилей, мешающих движению. Водители жалуются, что камеры фиксируют их маневры как нарушение правил дорожного движения, но не штрафуют нарушителей, оставляющих машины на крайних полосах. МВД пояснило, что такие штрафы выписываются автоматически, без учета препятствий на дороге. В случае несогласия с предписанием водители могут его обжаловать через портал eGov.kz, qamqor.gov.kz или обратиться в отдел административной практики УАП ДП Алматы. В ведомстве также подчеркнули, что нарушения правил парковки караются штрафом в размере 10 МРП, однако эвакуация автомобилей, затрудняющих движение, в городе практически не проводится. Юристы рекомендуют водителям подавать жалобы в течение 25 дней, если штраф был выписан несправедливо.",
   "Министр энергетики Алмасадам Саткалиев сообщил, что ежегодный рост потребления электроэнергии в Казахстане составляет в среднем 4%. Это связано с улучшением качества жизни, ростом промышленного производства, развитием социально-бытового сектора и малого и среднего бизнеса. В регионах, где произошла смена часового пояса, темпы роста потребления соответствуют средним показателям, в то время как в регионах без перевода времени рост выше. Например, в Западно-Казахстанской области потребление электроэнергии увеличилось на 10%. Данные подтверждаются статистической отчетностью системного оператора KEGOC.",
   "06 марта 2025 года в Мажилисе Парламента состоялось заседание Рабочей группы Министерства энергетики по вопросам развития внутристрановой ценности и поддержки отечественных товаропроизводителей в нефтегазовых проектах. В заседании приняли участие депутаты Парламента, государственные органы и представители бизнеса. Вице-министр энергетики К. Арымбек, председатель Правления АО НК «КазМунайГаз» А. Хасенов и Генеральный директор ТОО «Самрук-Казына Контракт» Э. Доскенов представили доклады о развитии местного содержания в закупках товаров, работ и услуг для недропользователей. В рамках работы пересматриваются Программы развития внутристрановой ценности, вводятся целевые индикаторы увеличения доли местного содержания, а также новые механизмы закупок для поддержки казахстанских поставщиков. Министерство энергетики совместно с депутатами Парламента внесло изменения в Кодекс о недрах, устанавливающие обязательную минимальную долю местного содержания в контрактах на недропользование. Также изменены Правила закупок недропользователей и их подрядчиков для стимулирования казахстанских производителей. Министерство энергетики продолжит работу в данном направлении."
]

new_texts_clean = [natasha_lemmatize(txt) for txt in new_texts]

preds = best_pipeline.predict(new_texts_clean)

for txt, pred in zip(new_texts, preds):
    label_str = "Минторговля" if pred == 1 else "Другое министерство"
    print(f"\nТекст: {txt}\n→ Класс: {label_str}")


Текст: Постановлением правительства Асет Ильясович Нусупов назначен на должность вице-министра торговли и интеграции Республики Казахстан. Об этом сообщает пресс-служба министерства.

Асет Нусупов родился в 1983 году в Алматы, окончил Казахский национальный университет им. аль-Фараби и Иллинойский институт технологий (США). Начал карьеру в 2004 году менеджером АО «Национальный инновационный фонд».

В разное время работал в инвестиционных и финансовых структурах, включая Департамент финансирования венчурных проектов Flagship Ventures (США), Small Enterprise Assistance Funds (США), Инвестиционный фонд Казахстана, представительства Всемирного банка и Азиатского банка развития в Центральной Азии. Также занимал руководящие должности в Совете безопасности Республики Казахстан, где работал с 2010 по 2023 год.

До назначения на новую должность являлся заместителем заведующего отделом актуальных вопросов безопасности Совета безопасности РК.
→ Класс: Минторговля

Текст: Автовладельцы Алматы сто